---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

# the passed model, AUC over 73.5%

In [1]:

import pandas as pd
import numpy as np

def blight_model():
    from sklearn.model_selection import train_test_split    

    from sklearn.metrics import roc_curve, auc

    col_drop=['payment_amount' ,'agency_name','payment_date' ,
          'payment_status' ,'balance_due', 'collection_status',
          'inspector_name','violator_name' , 'grafitti_status',
          'violation_zip_code','mailing_address_str_number', 'mailing_address_str_name',  
          'zip_code', 'non_us_str_code','city', 'state', 'country',
         'disposition', 'violation_street_name','violation_description','compliance_detail',
         'ticket_issued_date','hearing_date','violation_code']
    
    df=pd.read_csv('train.csv',encoding='ISO-8859-1').drop(col_drop,axis=1).dropna()
    x_train=df.iloc[: , :-1]
    y_train=df.iloc[:,-1]


    
    test_drop=['agency_name','inspector_name','violator_name' , 'grafitti_status',
          'violation_zip_code','mailing_address_str_number', 'mailing_address_str_name',  
          'zip_code', 'non_us_str_code','city', 'state', 'country',
         'disposition', 'violation_street_name','violation_description',
         'ticket_issued_date','hearing_date','violation_code']
    
    test=pd.read_csv('test.csv',encoding='ISO-8859-1').drop(test_drop,axis=1)


    
    from sklearn.preprocessing import MinMaxScaler
    scale = MinMaxScaler()

    scale_train = scale.fit_transform(x_train)

    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier( n_estimators=200  , criterion='entropy' , max_depth=3 ,bootstrap= False, class_weight='balanced',
                                 random_state=0).fit(scale_train, y_train)

    
    

    predictions=clf.predict(scale.transform(test))

    
    return pd.Series(predictions,index=[test.ticket_id],name='compliance' )

'def blight_model():\n    \n    # Your code here\n    \n    return # Your answer here'

# my journey to build the perfect model 
## Continued in the other notebook

In [55]:
col_drop=['payment_amount' ,'agency_name','payment_date' ,
          'payment_status' ,'balance_due', 'collection_status',
          'inspector_name','violator_name' , 'grafitti_status',
          'violation_zip_code','mailing_address_str_number', 'mailing_address_str_name',  
          'zip_code', 'non_us_str_code','city', 'state', 'country',
         'disposition', 'violation_street_name','violation_description','compliance_detail',
         'ticket_issued_date','hearing_date','violation_code']






df=pd.read_csv('train.csv',encoding='ISO-8859-1').drop(col_drop,axis=1).dropna()

df

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,violation_street_number,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,compliance
0,22056,2900.0,250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0
1,27586,4311.0,750.0,20.0,10.0,75.0,0.0,0.0,855.0,1.0
5,22046,6478.0,250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0
6,18738,8027.0,750.0,20.0,10.0,75.0,0.0,0.0,855.0,0.0
7,18735,8228.0,100.0,20.0,10.0,10.0,0.0,0.0,140.0,0.0
...,...,...,...,...,...,...,...,...,...,...
250278,285106,20009.0,200.0,20.0,10.0,20.0,0.0,0.0,250.0,0.0
250287,284650,15725.0,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,0.0
250288,285125,7152.0,500.0,20.0,10.0,50.0,0.0,0.0,580.0,0.0
250290,284881,17403.0,200.0,20.0,10.0,0.0,0.0,0.0,230.0,1.0


In [56]:
x_train=df.iloc[: , :-1]
y_train=df.iloc[:,-1]

In [85]:
# using random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

traindata , testdata , trainlabel , testlabel = train_test_split( x_train , y_train)
clf = RandomForestClassifier(n_estimators=300 ,max_depth=5, random_state = 0 )
clf.fit(traindata, trainlabel)

pred = clf.predict(testdata)
fpr, tpr, _ = roc_curve(testlabel, pred)
roc_auc = auc(fpr, tpr)
roc_auc


In [121]:
# using GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc

clf = GradientBoostingClassifier(  learning_rate=1 , max_depth=4 ,  n_estimators=1000 ,random_state=0 )
clf.fit(traindata, trainlabel)


pred = clf.predict(testdata)
fpr, tpr, _ = roc_curve(testlabel, pred)
roc_auc = auc(fpr, tpr)
roc_auc



0.5409797798322611

In [115]:
# using svc
from sklearn.svm import SVC

from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()

scale_train = scale.fit_transform(traindata)
scale_test = scale.transform(testdata)

clf = SVC( kernel='rbf', C=0.1, random_state= 0)

pred= clf.fit(scale_train, trainlabel).decision_function(scale_test)



#pred = clf.predict(scale_test)
fpr, tpr, _ = roc_curve(testlabel, pred)
roc_auc = auc(fpr, tpr)
roc_auc


0.5689198286497108

In [118]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()

scale_train = scale.fit_transform(traindata)
scale_test = scale.transform(testdata)

nnclf = MLPClassifier(hidden_layer_sizes = [100, 200 , 300], solver='adam', random_state = 0).fit(scale_train, trainlabel)
pred=nnclf.predict(scale_test)
fpr, tpr, _ = roc_curve(testlabel, pred)
roc_auc = auc(fpr, tpr)
roc_auc


0.5583784444116465

In [63]:
test_drop=['agency_name','inspector_name','violator_name' , 'grafitti_status',
          'violation_zip_code','mailing_address_str_number', 'mailing_address_str_name',  
          'zip_code', 'non_us_str_code','city', 'state', 'country',
         'disposition', 'violation_street_name','violation_description',
         'ticket_issued_date','hearing_date','violation_code']
test=pd.read_csv('test.csv',encoding='ISO-8859-1').drop(test_drop,axis=1)


test.shape

(61001, 9)

In [119]:
predictions=clf.predict(test)
predictions.shape

KeyboardInterrupt: 

In [ ]:
f=pd.Series(predictions,index=[test.ticket_id],name='compliance' )
f

In [97]:
f.unique()

array([1., 0.])

In [120]:
predictions=nnclf.predict(test)
f=pd.Series(predictions,index=[test.ticket_id],name='compliance' )
f

ticket_id
284932       1.0
285362       1.0
285361       1.0
285338       1.0
285346       1.0
            ... 
376496       1.0
376497       1.0
376499       1.0
376500       1.0
369851       1.0
Name: compliance, Length: 61001, dtype: float64